In [23]:
import pandas as pd
from io import StringIO
import os
from kafka import KafkaProducer
import json
from time import sleep
import boto3

In [25]:
# Configurar o produtor Kafka
producer = KafkaProducer(
    bootstrap_servers=['awari-kafka:9093'], # Altere para o endereço do seu servidor Kafka
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

NoBrokersAvailable: NoBrokersAvailable

In [19]:
# Caminhos para os arquivos CSV
estado_csv_path = '../Exercicio-07/data/estados.csv'
municipio_csv_path = '../Exercicio-07/data/municipios.csv'
base_path = '../Exercicio-07/UF' #Enviar o arquivo final

In [20]:
# Ler os arquivos CSV
df_estados = pd.read_csv(estado_csv_path)
df_municipios = pd.read_csv(municipio_csv_path)

In [21]:
# Merge dos DataFrames de estados e municípios com base em 'codigo_uf'
df_completo = pd.merge(df_municipios, df_estados, how='left', left_on='codigo_uf', right_on='codigo_uf')
# Selecionar e renomear as colunas relevantes para o novo DataFrame
df_completo = df_completo[['nome_x', 'nome_y', 'uf']]
df_completo.columns = ['Cidade', 'Estado', 'Sigla']
df_completo

,Cidade,Estado,Sigla
0,Abadia de Goiás,Goiás,GO
1,Abadia dos Dourados,Minas Gerais,MG
2,Abadiânia,Goiás,GO
3,Abaeté,Minas Gerais,MG
4,Abaetetuba,Pará,PA
...,...,...,...
5565,Xique-Xique,Bahia,BA
5566,Zabelê,Paraíba,PB
5567,Zacarias,São Paulo,SP
5568,Zé Doca,Maranhão,MA


In [ ]:
# Inicializar o cliente MinIO
client = boto3.client(
    's3',
    endpoint_url='http://localhost:9000', # Altere para o endpoint do seu servidor MinIO
    aws_access_key_id='minioadmin',
    aws_secret_access_key='minioadmin',
    region_name='us-east-1'
)

In [ ]:
for sigla in df_completo['Sigla'].unique():
    # Criar a pasta do estado localmente (opcional se só for enviar ao MinIO)
    # estado_path = os.path.join(base_path, sigla)
    # os.makedirs(estado_path, exist_ok=True)

    # Filtrar os municípios do estado
    df_estado = df_completo[df_completo['Sigla'] == sigla]

    # Caminho do arquivo CSV local (opcional)
    # estado_csv_path = os.path.join(estado_path, 'cidades.csv')

    # Salvar os municípios do estado em um arquivo CSV local (opcional)
    df_estado.to_csv(estado_csv_path, index=False)

    # Criar um buffer em memória para o CSV
    csv_buffer = StringIO()
    df_estado.to_csv(csv_buffer, index=False)

    # Caminho do arquivo no MinIO
    minio_key = f'Exercicio07/{sigla}/cidades.csv'

    # Enviar o arquivo CSV para o MinIO
    client.put_object(Bucket='exercicio07', Key=minio_key, Body=csv_buffer.getvalue())
    print(f'Arquivo enviado para {minio_key}')

    # Enviar uma mensagem para o Kafka
    message = {'Sigla': sigla, 'Arquivo': minio_key}
    producer.send('minio-uploads', value=message)
    print(f'Mensagem enviada para o Kafka: {message}')

    # Aguardar 5 segundos antes de enviar o próximo arquivo
    sleep(5)

# Fechar o produtor Kafka
producer.close()